# DataNeeded

## 1 預處理

In [1]:
import pandas as pd
import json
import os
import re

In [2]:
# 讀取數據文件
Adata = pd.read_csv("[2] 2019320fin.csv", encoding="utf-8-sig")

Adata.head()

,id,city,ctripName,lon,lat,value,stars,price,isquality,strategymedal,...,差评,有图片,商务出差,朋友出游,情侣出游,家庭亲子,独自出行,代人预订,其他,房型类数
0,345025,北京,北京中奥马哥孛罗大酒店,116.408622,40.005020,4.8,5,926,1,0,...,192,217,3512.0,345.0,229.0,509.0,332.0,212.0,309.0,6.0
1,345029,北京,北京西单美爵酒店,116.374998,39.903426,4.5,4,788,0,0,...,174,275,2405.0,262.0,303.0,1068.0,169.0,136.0,244.0,14.0
2,346488,北京,北京北方佳苑饭店,116.412954,39.910324,4.3,4,528,1,0,...,304,203,1746.0,302.0,222.0,1050.0,203.0,161.0,416.0,10.0
3,347308,北京,北京王府井希尔顿酒店,116.413340,39.914840,4.6,0,1321,0,0,...,157,458,1536.0,403.0,381.0,1578.0,216.0,98.0,239.0,15.0
4,347310,北京,北京长城饭店(原喜来登长城饭店),116.463587,39.944516,4.4,5,758,1,0,...,605,404,4893.0,459.0,352.0,548.0,353.0,269.0,369.0,15.0



| column   name  | describe                                 | \|   | column name        | describe                                           |
| -------------- | ---------------------------------------- | ---- | ------------------ | -------------------------------------------------- |
| id             | 酒店編号(base ctrip)                     | \|   | input_QunarName    | 輸入搜索的酒店名                                   |
| city           | 所在熱門城市                             | \|   | get_qunarName      | 在Qunar平臺檢索後的酒店名                          |
| ctripArea      | 酒店所在區域(ctrip)                      | \|   | qunarValue         | 酒店評分(Qunar)                                    |
| ctripName      | 酒店名稱(ctrip)                          | \|   | qunarCommentsCount | 酒店評論數(Qunar)                                  |
| value          | 酒店總評分(ctrip)                        | \|   | qunarURL           | 酒店網址(Qunar)                                    |
| stars          | 國家評定等級(ctrip)                      | \|   | fliggy_inputName   | 輸入搜索的酒店名                                   |
| price          | 酒店價格(ctrip 2018-11-24)               | \|   | fliggyName         | 在Fliggy平臺檢索後的酒店名                         |
| isquality      | 是否品質保證(ctrip)                      | \|   | diffValue          | fliggy_inputName &   fliggy_inputName 的文本值差異 |
| strategymedal  | 是否特牌(ctrip)                          | \|   | isTrust            | 是否信用入住(fliggy)                               |
| goldmedal      | 是否金牌(ctrip)                          | \|   | fliggyPrice        | 酒店價格(fliggy)                                   |
| silvermedal    | 是否銀牌(ctrip)                          | \|   | fliggyValue        | 酒店評分(fliggy)                                   |
| diamondRank    | 攜程評定鑽石等級(ctrip)                  | \|   | fliggyvolume       | 酒店評論數量(fliggy)                               |
| judgementscore | 用戶推薦入住百分比(ctrip 2018-11-24)     | \|   | fliggyURL          | 網址(fliggy)                                       |
| commentscount  | 酒店評論數(ctrip 2018-11-24)             | \|   |                    |                                                    |
| labelscount    | 區域或酒店類型標籤數量(ctrip 2018-11-24) | \|   |                    |                                                    |
| strlabels      | 區域或酒店類型標籤內容(ctrip 2018-11-24) | \|   |                    |                                                    |
| servicescount  | 酒店服務項目數量(ctrip)                  | \|   |                    |                                                    |
| strservices    | 酒店服務項目內容                         | \|   |                    |                                                    |
| ctripURL       | 酒店網址(ctrip)                          | \|   |                    |                                                    |

---

## mathcing 2 

Exact Match X1 城市、X4星级，然后找价格在实验组价格的上下10%区间里的所有酒店，计算他们Diff_Rating=CtripRating-QunarRating, 然后选择Diff_Rating为中位数的哪一家作为match的对照组。

<font color="blue">- 攜程選優酒店 Lf ( 初步定義: 持 strategymedal的酒店)</font>

In [3]:
# 特徵選擇
selFeature = [
    "id",
    "city",
    "ctripName",
    "stars",
    "strategymedal",
    "goldmedal",
    "silvermedal",
    "price",
    "value",
    "qunarValue",
]


In [4]:
# 獲取優選酒店
# Lf = Adata[Adata.strategymedal == 1]
# 重新定义 優選酒店
Lf = Adata[(Adata.strategymedal + Adata.goldmedal + Adata.silvermedal) != 0]
# 优选酒店数量
rre = Lf.strategymedal.shape[0]
Lf = Lf[selFeature]
Lf.sort_values("id", inplace=True)
Lf["diffValue"]=Lf.value-Lf.qunarValue
Lf.reset_index(inplace=True, drop=True)
print(Lf.shape)
Lf.head(3)

(2972, 11)


,id,city,ctripName,stars,strategymedal,goldmedal,silvermedal,price,value,qunarValue,diffValue
0,344928,三亚,三亚亚龙湾万豪度假酒店,5,0,1,0,951,4.6,4.6,0.0
1,345001,上海,上海日航饭店,5,0,0,1,970,4.4,4.3,0.1
2,345008,广州,广州方圆奥克伍德豪景,0,0,1,0,1040,4.5,4.6,-0.1


- <font color="blue"> 攜程非選優酒店 Rf

In [5]:
Rf = Adata[(Adata.strategymedal + Adata.goldmedal + Adata.silvermedal) == 0]
Rf = Rf[selFeature]
Rf.sort_values("id", inplace=True)
Rf["diffValue"]=Rf.value-Rf.qunarValue
Rf.reset_index(inplace=True, drop=True)
print(Rf.shape)
Rf.head()

(12915, 11)


,id,city,ctripName,stars,strategymedal,goldmedal,silvermedal,price,value,qunarValue,diffValue
0,141587,深圳,深雅酒店(深圳时尚店),0,0,0,0,121,2.9,3.0,-0.1
1,344612,南京,南京卡特快捷宾馆(原摩卡庄园快捷酒店),0,0,0,0,158,3.0,3.4,-0.4
2,344853,广州,广州富力君悦大酒店,5,0,0,0,1269,4.6,4.5,0.1
3,344857,广州,广州花园酒店,5,0,0,0,1166,4.6,4.6,0.0
4,344881,广州,广州中国大酒店,5,0,0,0,902,4.5,4.5,0.0


- <font color="blue">數據文件輸出

In [6]:
Lf.to_csv(r"H1\H1 Lf Ctrip and Qunar.csv",encoding="utf-8-sig")
Rf.to_csv(r"H1\H1 Rf Ctrip and Qunar.csv",encoding="utf-8-sig")

- match

In [16]:
#from math import radians, cos, sin, asin, sqrt
import pandas as pd
import json
import os
import re
import numpy as np


Lf = pd.read_csv(r"H1\H1 Lf Ctrip and Qunar.csv", encoding="utf-8-sig")
RRf = pd.read_csv(r"H1\H1 Rf Ctrip and Qunar.csv", encoding="utf-8-sig")


# ----------------------------------------------------hard match
num = 0
resultS = []
resultD = []
k = 1
for _ in Lf.id:
    Rf = RRf.copy()
    # match 所在城市
    Rf = Rf[Rf.city == Lf.loc[num].city]
    # match 酒店星级
    Rf = Rf[Rf.stars == Lf.loc[num].stars]
    t = Lf.loc[num].price

    # 价格差
    Rf = Rf[(abs(Rf.price-t))/t <= 0.1]
    if Rf.shape[0] == 0:
        num += 1
        continue
    # 将配对到的数量存起来
    resultS.append(Rf.shape[0])
    # 携程与去哪的评分差值排序
    Rf.sort_values(["diffValue"],inplace=True)
    
    ind=Rf["diffValue"].shape[0]//2
    # 取出中位数
    Rf=Rf.iloc[ind:ind+1]

#     Rf["matching to id"] = Lf.loc[num].id
#     Rf["matching to name"] = Lf.loc[num].ctripName
#     print(Rf.id.tolist()[0])
    t = {
        "matchingNum": [k, k],
        "matchingLabel": [1, 0],
        "id": [Lf.loc[num].id, Rf.id.tolist()[0]],
        "city": [Lf.loc[num].city, Rf.city.tolist()[0]],
        "hotelname": [Lf.loc[num].ctripName, Rf.ctripName.tolist()[0]],
        "stars": [Lf.loc[num].stars, Rf.stars.tolist()[0]],
        "hotelprice": [Lf.loc[num].price, Rf.price.tolist()[0]],
        "diffValue":[Lf["diffValue"].tolist()[0],Rf["diffValue"].tolist()[0]],
        "ValueMedian": ["", ind]
    }

    num += 1
    k += 1
    resultD.append(pd.DataFrame(t))


resultD[0]

,matchingNum,matchingLabel,id,city,hotelname,stars,hotelprice,diffValue,ValueMedian
0,1,1,345001,上海,上海日航饭店,5,970,0.0,
1,1,0,374924,上海,上海新发展亚太JW万豪酒店,5,958,0.1,5


In [8]:
t1=resultS.count(0)
t2=len(resultS)
t3=max(resultS)
str1=f"匹配失败数量:{t2-t1},匹配失败数量:{t1},匹配最多数量:{t3}"

# print(str1,"\n",resultS)

In [9]:
# 將所有匹配到的酒店合併成一個DF 對象
fa = pd.concat(resultD, axis=0, sort=False)
fa=fa.reset_index(drop="index")
sete=[]
for i in range(fa.shape[0]):
    D=Adata[Adata.id==fa.iloc[i].id]
    sete.append(D)

In [10]:
fa.columns.tolist()

['matchingNum',
 'matchingLabel',
 'id',
 'city',
 'hotelname',
 'stars',
 'hotelprice',
 'diffValue',
 'ValueMedian']

In [11]:
matchinset=pd.concat(sete,axis=0)
matchinset.reset_index(drop="index",inplace=True)

In [12]:
outfile=pd.concat([matchinset,fa[["matchingNum","matchingLabel","diffValue","ValueMedian"]]],axis=1)

In [13]:
outfile.to_csv("[3] matchHotel.csv",encoding="utf-8-sig")

In [15]:
filename="[3] matchHotel"

## 输出match 酒店

In [17]:
def tocsv2(filename):
    tager=pd.read_csv(f"{filename}.csv",encoding="utf-8-sig")

    x=tager.iloc[0::2,1:]
    x.reset_index(drop="index",inplace=True)

    y=tager.iloc[1::2,1:]
    y.columns=[(i+"_matched Result") for i in y.columns]
    y.reset_index(drop="index",inplace=True)

    z=pd.concat([x,y],axis=1)
    z.to_csv(f"{filename}_.csv",encoding="utf-8-sig")
    
tocsv2("[3] matchHotel")

---

## **在此 补充包含配对酒店的评论数据表文件夹 rating后,再进行下一步的操作

In [18]:
import os
filecsv = pd.read_csv("[3] matchHotel.csv", encoding="utf-8-sig")

def getDF(id):
    filename = f"{str(id)}.csv"
    with open("rating\\"+filename, "r", encoding="utf-8-sig") as f:
        ret = ""
        for i in range(200):
            ret += (f.readline())
        try:
            a = ret.split("commentAccount")
            if len(a) > 1:
                with open("rating2\\"+filename, "w", encoding="utf-8-sig") as fw:
                    fw.write("commentAccount" + a[1])
                return 0
            else:
                return id
        except:
            print(filename)


yy = list(set(filecsv.id.tolist()))
# print(len(yy))
re=[]
for i in yy:
    a=getDF(i)
    if a!=0:
        re.append(a)

## 处理并输出统计前100条评论标准差的数据

In [17]:
filecsv = pd.read_csv("[3] matchHotel.csv", encoding="utf-8-sig")
listname=["id","city","ctripName","matchingNum","matchingLabel","diffValue","ValueMedian","value","stars","price"]
filecsv=filecsv[listname]
STD = []

# 将评论中前100条数据取出
for i in filecsv.id:
    try:
        t = pd.read_csv(f"{os.getcwd()}\\rating2\\{i}.csv",
                        encoding="utf-8-sig")
        STD.append(t.indiRating[:100].std())
        
    except:
        STD.append("NaN")
        
# 插入新的数据 STD 标准差     


In [19]:
filecsv.insert(8,column="STD",value=STD)


ValueError: cannot insert STD, already exists

- 输出数据 

In [20]:
filecsv

,id,city,ctripName,matchingNum,matchingLabel,diffValue,ValueMedian,value,STD,stars,price
0,345001,上海,上海日航饭店,1,1,0.0,NaN,4.4,1.16969,5,970
1,374924,上海,上海新发展亚太JW万豪酒店,1,0,0.1,5.0,4.6,0.94654,5,958
2,345008,广州,广州方圆奥克伍德豪景,2,1,0.0,NaN,4.5,0.771133,0,1040
3,456162,广州,广州雅诗阁服务公寓,2,0,0.1,6.0,4.5,0.836527,0,1071
4,345012,广州,桔子水晶酒店(广州淘金店),3,1,0.0,NaN,4.8,0.67577,0,826
5,512410,广州,广州船舶太古酒店,3,0,0.0,10.0,4.3,1.12845,0,883
6,345025,北京,北京中奥马哥孛罗大酒店,4,1,0.0,NaN,4.8,1.02555,5,926
7,374795,北京,北京伯豪瑞廷酒店,4,0,0.1,3.0,4.6,1.24048,5,936
8,345029,北京,北京西单美爵酒店,5,1,0.0,NaN,4.5,0.94313,4,788
9,638075,北京,北京金码大酒店,5,0,0.0,7.0,4.5,0.813947,4,760


In [21]:
filecsv.to_csv("[4] H1 matching with STD.csv",encoding="utf-8-sig")

In [22]:
def tocsv2(filename):
    tager=pd.read_csv(f"{filename}.csv",encoding="utf-8-sig")

    x=tager.iloc[0::2,1:]
    x.reset_index(drop="index",inplace=True)

    y=tager.iloc[1::2,1:]
    y.columns=[(i+"_matched Result") for i in y.columns]
    y.reset_index(drop="index",inplace=True)

    z=pd.concat([x,y],axis=1)
    z.to_csv(f"{filename}_.csv",encoding="utf-8-sig")
    
tocsv2("[4] H1 matching with STD")

## 输出包含所有评论的数据

In [10]:
SerTD = []
H1data = pd.read_csv("[4] H1 matching with STD.csv", encoding="utf-8-sig")
# 去除无效数据
H1data=H1data.iloc[:,2:]
H1data.head()
listname=["id","city","ctripName","matchingNum","matchingLabel","diffValue","ValueMedian","value","stars","price","STD"]
H1data=H1data[listname]
H1data

,id,city,ctripName,matchingNum,matchingLabel,diffValue,ValueMedian,value,stars,price,STD
0,345001,上海,上海日航饭店,1,1,0.0,NaN,4.4,5,970,1.169691
1,374924,上海,上海新发展亚太JW万豪酒店,1,0,0.1,5.0,4.6,5,958,0.946540
2,345008,广州,广州方圆奥克伍德豪景,2,1,0.0,NaN,4.5,0,1040,0.771133
3,456162,广州,广州雅诗阁服务公寓,2,0,0.1,6.0,4.5,0,1071,0.836527
4,345012,广州,桔子水晶酒店(广州淘金店),3,1,0.0,NaN,4.8,0,826,0.675770
5,512410,广州,广州船舶太古酒店,3,0,0.0,10.0,4.3,0,883,1.128449
6,345025,北京,北京中奥马哥孛罗大酒店,4,1,0.0,NaN,4.8,5,926,1.025550
7,374795,北京,北京伯豪瑞廷酒店,4,0,0.1,3.0,4.6,5,936,1.240481
8,345029,北京,北京西单美爵酒店,5,1,0.0,NaN,4.5,4,788,0.943130
9,638075,北京,北京金码大酒店,5,0,0.0,7.0,4.5,4,760,0.813947


In [11]:
g1=H1data.id.tolist()
for i in range(len(g1)):
    try:
        t = pd.read_csv(f"{os.getcwd()}\\rating2\\{g1[i]}.csv", encoding="utf-8-sig")
    except:
        a = [H1data.iloc[i:i+1, :]]
        dfe = pd.concat(a, axis=0)
        dfe.insert(5, column="用户评分", value=["NaN"])
        dfe.insert(5, column="用户序号", value=["NaN"])
        SerTD.append(dfe)


    a = [H1data.iloc[i:i+1, :] for _ in range(t.indiRating[:100].shape[0])]
    dfe = pd.concat(a, axis=0)

    dfe.insert(5, column="用户评分", value=(t.indiRating[:100]).tolist())
    dfe.insert(5, column="用户序号", value=[j+1 for j in range(t.indiRating[:100].shape[0])])
    if i%500==0:
        print(i)
    SerTD.append(dfe)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500


In [12]:
dfe=pd.concat(SerTD,axis=0)
dfe.reset_index(drop="index",inplace=True)
dfe.head()

,id,city,ctripName,matchingNum,matchingLabel,用户序号,用户评分,diffValue,ValueMedian,value,stars,price,STD
0,345001,上海,上海日航饭店,1,1,1,4.5,0.0,NaN,4.4,5,970,1.169691
1,345001,上海,上海日航饭店,1,1,2,4.3,0.0,NaN,4.4,5,970,1.169691
2,345001,上海,上海日航饭店,1,1,3,3,0.0,NaN,4.4,5,970,1.169691
3,345001,上海,上海日航饭店,1,1,4,5,0.0,NaN,4.4,5,970,1.169691
4,345001,上海,上海日航饭店,1,1,5,3.8,0.0,NaN,4.4,5,970,1.169691


- 从上往下阅读

In [49]:
dfe.to_csv("[4] H1 matching with allrating.csv",encoding="utf-8-sig")

- 左右两边阅读

In [44]:
ret=[]
for i in range(1,int(dfe.matchingNum.max())+1):
    t = dfe[dfe.matchingNum == i]
    T = t[t.ctripName == t.iloc[1].ctripName]
    T.reset_index(drop="index", inplace=True)
    TT = t[t.ctripName != t.iloc[1].ctripName]
    TT.columns=[(str(i)+"RM") for i in TT.columns.tolist()]
    TT.reset_index(drop="index", inplace=True)
    temp=pd.concat([T, TT], axis=1)
    ret.append(temp)

In [46]:
matchingwithallrating=pd.concat(ret,sort=False, axis=0)
matchingwithallrating.reset_index(drop="index", inplace=True)

In [48]:
matchingwithallrating.to_csv("[4] H1 matching with allrating_marge.csv",encoding="utf-8-sig")

---